# Insper - Supercomputação - Prova Intermediária

Questão sobre GPU / THRUST.


**Questão 4: Normalização de Vetor com Thrust**

Na análise de dados e no processamento de sinais, normalizar um vetor é uma operação comum que ajusta a escala dos elementos para que o vetor tenha uma norma unitária (ou seja, a soma dos quadrados dos elementos é igual a 1). Essa tarefa se torna computacionalmente intensiva para vetores de grande dimensão, fazendo com que a paralelização em GPU seja uma solução atrativa.

**Objetivo**:

Complemente o código abaixo usando a biblioteca Thrust para normalizar um vetor grande. O código inicial cria um vetor com valores aleatórios. Você deve implementar as etapas para:

1. calcular a norma L2 do vetor,
2. dividir cada elemento do vetor por essa norma e
3. imprimir o resultado final.

**Código Pré-Pronto**:

Complete o código com a sua solução.

In [8]:
%%writefile gpu.cu

// *************************************************************************
//
//         IMPORTS NECESSÁRIOS DA THRUST
//
// *************************************************************************

#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform.h>
#include <thrust/reduce.h>
#include <thrust/functional.h>
#include <thrust/iterator/counting_iterator.h>
#include <thrust/random.h>
#include <iostream>
#include <cmath>

// Função para gerar números aleatórios
struct prg
{
    float a, b;

    __host__ __device__
    prg(float _a = 0.f, float _b = 1.f) : a(_a), b(_b) {}

    __host__ __device__
    float operator()(const unsigned int n) const
    {
        thrust::default_random_engine rng(n);  // Inicializa com semente única
        thrust::uniform_real_distribution<float> dist(a, b);
        return dist(rng);
    }
};

// Função para normalizar cada elemento pelo valor da norma
struct normalize
{
    float norm;

    normalize(float _norm) : norm(_norm) {}

    __host__ __device__
    float operator()(const float& x) const
    {
        return x / norm;
    }
};

int main() {
    const int N = 1000000; // Tamanho do vetor

    // Inicializa um vetor no host
    thrust::counting_iterator<unsigned int> index_sequence_begin(0);
    thrust::host_vector<float> h_vec(N);

    // Preenche o vetor com números aleatórios entre 1.0 e 2.0
    thrust::transform(index_sequence_begin,
                      index_sequence_begin + N,
                      h_vec.begin(),
                      prg(1.f, 2.f));

    // Exibe alguns valores do vetor original para verificação
    std::cout << "Vetor original (primeiros 20 elementos):" << std::endl;
    for (int i = 0; i < 20; i++) {
        std::cout << h_vec[i] << " ";
    }
    std::cout << std::endl;

    // Copia o vetor do host para o dispositivo
    thrust::device_vector<float> d_vec = h_vec;

    // Calcula a norma L2 no dispositivo
    float norm = std::sqrt(thrust::reduce(d_vec.begin(),
                                          d_vec.end(),
                                          0.0f,
                                          thrust::plus<float>()));

    // Imprime a norma calculada
    std::cout << "Norma calculada: " << norm << std::endl;

    // Normaliza o vetor
    thrust::transform(d_vec.begin(),
                      d_vec.end(),
                      d_vec.begin(),
                      normalize(norm));

    // Copia o vetor normalizado de volta para o host
    thrust::copy(d_vec.begin(), d_vec.end(), h_vec.begin());

    // Exibe alguns valores do vetor normalizado para verificação
    std::cout << "Vetor normalizado (primeiros 20 elementos):" << std::endl;
    for (int i = 0; i < 20; i++) {
        std::cout << h_vec[i] << " ";
    }
    std::cout << std::endl;

    return 0;
}


Overwriting gpu.cu


Compilando o código

In [9]:
!nvcc -arch=sm_75 -std=c++14 gpu.cu -o gpu

Exemplo de execução:

In [14]:
%%time
!./gpu

Vetor original (primeiros 20 elementos):
1.00002 1.00002 1.00004 1.00007 1.00009 1.00011 1.00013 1.00016 1.00018 1.0002 1.00022 1.00025 1.00027 1.00029 1.00031 1.00034 1.00036 1.00038 1.0004 1.00043 
Norma calculada: 1222.48
Vetor normalizado (primeiros 20 elementos):
0.00081803 0.00081803 0.000818048 0.000818066 0.000818085 0.000818103 0.000818121 0.00081814 0.000818158 0.000818177 0.000818195 0.000818213 0.000818232 0.00081825 0.000818269 0.000818287 0.000818305 0.000818324 0.000818342 0.00081836 
CPU times: user 10.7 ms, sys: 90 µs, total: 10.8 ms
Wall time: 507 ms
